In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import datetime
mpl.rcParams['figure.figsize'] = (13,8)
mpl.rcParams['text.usetex'] = False
mpl.rcParams['text.latex.preamble'] = r'\usepackage[cm]{sfmath}'
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'cm'
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10

# Load in data and process

In [42]:
# https://geoportal.statistics.gov.uk/datasets/nuts-level-1-january-2018-boundaries
regions = gpd.read_file('./NUTS_Level_1_(January_2018)_Boundaries/NUTS_Level_1_(January_2018)_Boundaries.shp')
regions['region'] = regions['nuts118nm'].str.replace(' \(England\)', '',regex=True)

# Taken from Oxford and Cambridge admissions websites
statistics = pd.read_csv('statistics.csv')
statistics['probability'] = 100*(1.0+statistics['CambridgeApplications'])/(2.0+statistics['CambridgeApplications']+statistics['OxfordApplications'])

geodf = regions.merge(statistics, left_on = 'region', right_on = 'Regions').explode()

# Transformation
def getPolyCoords(row, geom, coord_type):
    if coord_type == 'x':
        return list(row[geom].exterior.coords.xy[0])
    elif coord_type == 'y':
        return list(row[geom].exterior.coords.xy[1])

geodf['x'] = geodf.apply(getPolyCoords, geom = 'geometry', coord_type = 'x', axis = 1)
geodf['y'] = geodf.apply(getPolyCoords, geom = 'geometry', coord_type = 'y', axis = 1)
geodf.drop('geometry', axis = 1, inplace=True)


# Fraction of applicants who apply to Cambridge nationally

In [63]:
(1.0+statistics['CambridgeApplications'].sum())/(2.0+statistics['OxfordApplications'].sum()+statistics['CambridgeApplications'].sum())

0.47149551479448387

# Make interactive plot

In [91]:
# We are going to create an interactive bokeh plot
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, HoverTool, Slider, CustomJS
from bokeh.layouts import column

# Create source for bokeh plot
source = ColumnDataSource(geodf)

# Create palette
from matplotlib import colors
palette = [colors.to_hex(plt.cm.RdBu(f)) for f in np.arange(30)/29]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 35, high = 65)

# Create bokeh figure
tools = 'wheel_zoom,pan,reset,hover'
plot = figure(title = 'Fraction who choose Cambridge (%)', plot_height=700 , plot_width=450, toolbar_location='above', tools=tools, active_scroll="wheel_zoom")

#Add patch renderer to figure - these are the shapes of the postcode areas
plot.patches('x','y', source=source, fill_alpha=1, line_width=0.5, line_color='black', fill_color={'field':'probability' , 'transform':color_mapper})

plot.circle([4.5e5],[2e5], size=10, color="crimson", alpha=1.0)
plot.circle([5.4e5],[2.5e5], size=10, color="navy", alpha=1.0)

# Add the tooltips when you hover over a patch
hover = plot.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("Region", "@region"),("Cambridge","@{CambridgeApplications}"),("Oxford","@{OxfordApplications}"),("Fraction","@probability{0.2f} %")]

# Customise plot
plot.axis.visible = False
plot.outline_line_color = None
plot.xgrid.grid_line_color = None
plot.ygrid.grid_line_color = None

# Add colour bar
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=10, height=600,location='center', orientation='vertical')
plot.add_layout(color_bar, 'right')

# Combine plot and slider and output
layout = column(plot)
output_file("./oxbridgeregional.html")
show(layout)

In [75]:
geodf

objectid nuts118cd             nuts118nm   bng_e   bng_n     long  \
0  0         1       UKC  North East (England)  417313  600358 -1.72890   
   1         1       UKC  North East (England)  417313  600358 -1.72890   
1  0         2       UKD  North West (England)  350015  506280 -2.77237   
   1         2       UKD  North West (England)  350015  506280 -2.77237   
   2         2       UKD  North West (England)  350015  506280 -2.77237   
...        ...       ...                   ...     ...     ...      ...   
11 3        12       UKN      Northern Ireland   86601  535325 -6.85481   
   4        12       UKN      Northern Ireland   86601  535325 -6.85481   
   5        12       UKN      Northern Ireland   86601  535325 -6.85481   
   6        12       UKN      Northern Ireland   86601  535325 -6.85481   
   7        12       UKN      Northern Ireland   86601  535325 -6.85481   

            lat    st_areasha    st_lengths            region  \
0  0  55.297031  8.609939e+09  6.575782e+05        North East   
   1  55.297031  8.609939e+09  6.575782e+05        North East   
1  0  54.449451  1.418261e+10  1.063053e+06        North West   
   1  54.449451  1.418261e+10  1.063053e+06        North West   
   2  54.449451  1.418261e+10  1.063053e+06        North West   
...         ...           ...           ...               ...   
11 3  54.614941  1.433563e+10  8.329563e+05  Northern Ireland   
   4  54.614941  1.433563e+10  8.329563e+05  Northern Ireland   
   5  54.614941  1.433563e+10  8.329563e+05  Northern Ireland   
   6  54.614941  1.433563e+10  8.329563e+05  Northern Ireland   
   7  54.614941  1.433563e+10  8.329563e+05  Northern Ireland   

               Regions  OxfordApplications  OxfordOffers  OxfordAdmitted  \
0  0        North East                 803           192             162   
   1        North East                 803           192             162   
1  0        North West                3135           735             623   
   1        North West                3135           735             623   
   2        North West                3135           735             623   
...                ...                 ...           ...             ...   
11 3  Northern Ireland                 399            68              59   
   4  Northern Ireland                 399            68              59   
   5  Northern Ireland                 399            68              59   
   6  Northern Ireland                 399            68              59   
   7  Northern Ireland                 399            68              59   

      CambridgeApplications  CambridgeOffers  CambridgeAdmitted  probability  \
0  0                    921              300                228    53.418308   
   1                    921              300                228    53.418308   
1  0                   2612              737                547    45.451383   
   1                   2612              737                547    45.451383   
   2                   2612              737                547    45.451383   
...                     ...              ...                ...          ...   
11 3                    460              131                111    53.542393   
   4                    460              131                111    53.542393   
   5                    460              131                111    53.542393   
   6                    460              131                111    53.542393   
   7                    460              131                111    53.542393   

                                                      x  \
0  0  [397942.9308000002, 400000.0, 400804.831799999...   
   1  [413901.89859999996, 414032.89609999955, 41252...   
1  0  [357136.20299999975, 356937.8021, 358216.50389...   
   1  [321160.1980999997, 321786.89690000005, 323317...   
   2  [332100.3691999996, 331637.10450000037, 331701...   
...                                                 ...   
11 3  [173134.65079999994, 172956.050